In [1]:
from probability.pandas import DiscreteDistribution, ConditionalTable

# Bayesian Reasoning and Machine Learning

### 1.1.1 Interpreting Conditional Probability

In [2]:
darts = DiscreteDistribution.from_dict(
    data={i: 1/20 for i in range(1, 21)},
    var_names='region'
)
print(darts)

DiscreteDistribution: P(region)
region
1     0.05
2     0.05
3     0.05
4     0.05
5     0.05
6     0.05
7     0.05
8     0.05
9     0.05
10    0.05
11    0.05
12    0.05
13    0.05
14    0.05
15    0.05
16    0.05
17    0.05
18    0.05
19    0.05
20    0.05
Name: p, dtype: float64
sum: 1.0000000000000002


In [3]:
darts.given(region__ne=20).p(region=5)

0.05263157894736841

In [4]:
darts.p(region=5, region__ne=20) / darts.p(region__ne=20)

0.05263157894736841

In [5]:
darts.p(region=5) / darts.p(region__ne=20)

0.05263157894736841

In [6]:
1 / 19

0.05263157894736842

### 1.1.2 Probability Tables

In [7]:
p_c = DiscreteDistribution.from_counts({
    'england': 60_776_238,
    'scotland': 5_116_900,
    'wales': 2_980_700
}, 'country')
print(p_c)

DiscreteDistribution: P(country)
country
england     0.882429
scotland    0.074294
wales       0.043278
Name: p, dtype: float64
sum: 0.9999999999999999


In [8]:
p_l__c = ConditionalTable.from_dict({
    ('english', 'england'): 0.95,
    ('english', 'scotland'): 0.7,
    ('english', 'wales'): 0.6,
    ('scottish', 'england'): 0.04,
    ('scottish', 'scotland'): 0.3,
    ('scottish', 'wales'): 0.0,
    ('welsh', 'england'): 0.01,
    ('welsh', 'scotland'): 0.0,
    ('welsh', 'wales'): 0.4,
}, ['language', 'country'], 'country')
print(p_l__c)

ConditionalTable: P(language|country)
language  country 
english   england     0.95
          scotland    0.70
          wales       0.60
scottish  england     0.04
          scotland    0.30
          wales       0.00
welsh     england     0.01
          scotland    0.00
          wales       0.40
Name: p, dtype: float64
sum: 3.0


In [10]:
p_lc = p_l__c * p_c
print(p_lc)

DiscreteDistribution: P(language,country)
language  country 
english   england     0.838307
          scotland    0.052006
          wales       0.025967
scottish  england     0.035297
          scotland    0.022288
          wales       0.000000
welsh     england     0.008824
          scotland    0.000000
          wales       0.017311
Name: p, dtype: float64
sum: 0.9999999999999999


1.3.1 Two dice : what were the individual scores?